In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io #Used to load the OCTAVE *.mat files
import scipy.misc #Used to show matrix as an image
import matplotlib.cm as cm #Used to display images in a specific colormap
import random #To pick random images to display
import math
import itertools
from scipy.special import expit #Vectorized sigmoid function
import array


In [2]:
dataFile = "data/ex4data1.mat"
data = scipy.io.loadmat(dataFile)


# X has 5000 examples and 400 features making it 5000x400
X = data['X']
y = data['y']

# training set / test set code
# shuffling data set because later on pd.get_dummies needs to identify 10 unique elements but can't if the data only contains 
# classifications for the first 6 digits
combined = np.append(X, y, axis=1)
np.random.shuffle(combined)

yComb = combined[:,400]
yComb = yComb.reshape(len(yComb),1)

Xcomb = combined[:,0:400]
Xcomb = Xcomb.reshape(len(Xcomb),400)

# will be using 70% of data to train hence 3500 x 400
Xtrain = Xcomb[0:3500,:]
Xtrain = np.insert(Xtrain, 0, 1, axis=1)
yTrain = yComb[0:3500]

# then the test set will be 1500x400
Xtest = Xcomb[3500:5001,:]
yTest = yComb[3500:5000]

#end of train/test set code

X = np.insert(X, 0, 1, axis = 1)



In [3]:
# taken from ex3
def reshapeRow(row):
    """
    @param {row} 1 x 401 matrix since an image of a digit is 20x20 + 1 that was added as a bias
    Function that takes in the pixel intensity values and puts it into a 20x20 square 
    """ 
    # the [1:] is used to take everything after the 1st index 
    
    return row[1:].reshape(20,20).T

def displayData(indiciesToDisplay = None):
    """
    Function that selects 100 random examples for the 5000 we have and organizes
    them into a 10x10 matrix
    """
    width = 20
    height = 20
    numRows = 10
    numCols = 10
    
    if not indiciesToDisplay:
        indiciesToDisplay = random.sample(range(X.shape[0]), numRows * numCols)

    
    bigPicture = np.zeros((height * numRows, width * numCols))
    
    iRow = 0
    iCol = 0

    for i in indiciesToDisplay :
        if iCol == numCols:
            iCol = 0
            iRow += 1
 
        curImg = reshapeRow(X[i])
        bigPicture[iRow * height :iRow * height + curImg.shape[0], 
                    iCol * width : iCol * width + curImg.shape[1]] = curImg
        iCol += 1 
    fig = plt.figure( figsize = (6,6) )
    img = scipy.misc.toimage( bigPicture )
    plt.imshow(img,cmap = cm.Greys_r)

#displayData()

In [4]:
thetaFile = "data/ex4weights.mat"
thetas1 = scipy.io.loadmat(thetaFile)

# Theta1.shape = 25 x 401
Theta1 = thetas1['Theta1']

# Theta2.shape = 10 x 26
Theta2 = thetas1['Theta2']

thetaUnrolled = np.r_[Theta1.ravel(), Theta2.ravel()]

inputSize = 400
hiddenSize = 25
outputSize = 10


# m = 5000
m = X.shape[0]
# n = 401
n = X.shape[1]

aVals = [None] * 3
zVals = [None] * 2

In [5]:
# the following 2 functions are needed because fmin_cg passes in X as an unrolled vector
def reshape(X, m, n):
    return np.array(X).reshape((m,n))

def flatten(X):
    return np.array(X.flatten()).reshape((X.shape[0]*(X.shape[1]),1))


# used to flatted D1, D2, taken from kaleko github
def flattenParams(thetas_list):
    input_layer_size = 400
    hidden_layer_size = 25
    output_layer_size = 10
    """
    Hand this function a list of theta matrices, and it will flatten it
    into one long (n,1) shaped numpy array
    """
    flattened_list = [ mytheta.flatten() for mytheta in thetas_list ]
    combined = list(itertools.chain.from_iterable(flattened_list))
    assert len(combined) == (input_layer_size+1)*hidden_layer_size + \
                            (hidden_layer_size+1)*output_layer_size
    return np.array(combined).reshape((len(combined),1))

def sigmoid(X, theta):
    return expit(np.dot(X,theta))

def costFunction(thetas, X, y, lmbda):
    
    X = reshape(X,3500,401)
    theta1 = thetas[0:(hiddenSize*(inputSize+1))].reshape(hiddenSize,(inputSize+1))
    theta2 = thetas[(hiddenSize*(inputSize+1)):].reshape(10,(hiddenSize+1))
    
    m = X.shape[0]

    a1 = X # 5000 x 401
    z2 = theta1.dot(X.T)
    
    a2 = expit(z2) # 25 x 5000
    a2 = np.insert(a2,0,1,axis=0)
       
    z3 = theta2.dot(a2)
    a3 = expit(z3)
    
    aVals[0] = a1
    aVals[1] = a2
    aVals[2] = a3
    
    zVals[0] = z2
    zVals[1] = z3
     
    tempY = pd.get_dummies(y.ravel()).values
    
    J = (-1/m)*np.sum(np.log(a3.T)*(tempY)+np.log(1-a3).T*(1-tempY)) + \
    (lmbda/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))  
    
    return J

In [6]:
def testCost():
    J = costFunction(thetaUnrolled,Xtrain,yTrain,1)
    #print("Expected value for training set is ~ 1.83\nActual value %.9f"%J)

    #print("Expected value is ~ 0.383\nActual value %.9f"%J)
testCost()



In [7]:
def gradientSigmoid(z):
    return expit(z)*(1-expit(z))

def initRandomThetas():
    epsilon = 0.12
    theta1Size = (hiddenSize, inputSize + 1)
    theta2Size = (outputSize, hiddenSize + 1)
    
    
    theta1 = np.random.rand(*theta1Size).ravel()*2*epsilon - epsilon
    theta2 = np.random.rand(*theta2Size).ravel()*2*epsilon - epsilon
    
    
    thetas = np.r_[Theta1.ravel(), Theta2.ravel()]
    return thetas

temp = initRandomThetas()


In [8]:

def backProp(thetas,X,y, lmbda):
    X = reshape(X,3500,401)
    m = X.shape[0]
    
    theta1 = thetas[0:(hiddenSize*(inputSize+1))].reshape(hiddenSize,(inputSize+1))
    theta2 = thetas[(hiddenSize*(inputSize+1)):].reshape(10,(hiddenSize+1))
    
    a1 = aVals[0]
    a2 = aVals[1].T
    a3 = aVals[2].T
    
    z2 = zVals[0].T
    z3 = zVals[1].T
    
        
    D1 = np.zeros((theta1.shape))
    D2 = np.zeros((theta2.shape))
    
    for i in range(m):
        a1Cur = X[i]
        a2Cur = a2[:,1:][i]
        z2Cur = z2[i]
        a3Cur = a3[i]
        z3Cur = z3[i]
    
        a1Cur = a1Cur.reshape(len(a1Cur),1)
        a2Cur = a2Cur.reshape(len(a2Cur),1)
        z2Cur = z2Cur.reshape(len(z2Cur),1)
        a3Cur = a3Cur.reshape(len(a3Cur),1)
        z3Cur = z3Cur.reshape(len(z3Cur),1)
        
        yT = np.zeros((10,1))
        yT[int(y[i]) - 1] = 1
                
        delta3 = a3Cur - yT
        delta2 = theta2[:,1:].T.dot(delta3)*gradientSigmoid(z2Cur)
        
        
        a2Cur = np.insert(a2Cur, 0, 1, axis=0)

        D1 = D1 + delta2.dot(a1Cur.T)
                
        delta3 = delta3.reshape(len(delta3),1)
        D2 = D2 + delta3.dot(a2Cur.T)
        
    D1 = D1/m
    D2 = D2/m
    
    D1[:,1:] = D1[:,1:] + (float(lmbda)/m)*theta1[:,1:]
    D2[:,1:] = D2[:,1:] + (float(lmbda)/m)*theta2[:,1:]
    
    return flattenParams([D1, D2]).flatten()
       
def checkGradient(thetas,D,X,y,lmbda):
    epsilon = 0.0001
    xT = flatten(X)
    n_elems = len(thetas) 
    thetas = thetas.reshape(len(thetas),1)
    #Pick ten random elements, compute numerical gradient, compare to respective D's
    for i in range(10):
        x = int(np.random.rand()*X.shape[0])
        epsvec = np.zeros((n_elems,1))
        epsvec[x] = epsilon
        cost_high = costFunction(thetas + epsvec, X, y, lmbda)
        cost_low  = costFunction(thetas - epsvec, X, y, lmbda)
        mygrad = (cost_high - cost_low) / float(2*epsilon)
        print ("Element: %d. Numerical Gradient = %f. BackProp Gradient = %f."%(x,mygrad,D[x]))
        

Ds = backProp(np.r_[Theta1.ravel(),Theta2.ravel()],Xtrain,yTrain,0)
checkGradient(thetaUnrolled,Ds,Xtrain,yTrain,0)
        


Element: 1114. Numerical Gradient = 0.000190. BackProp Gradient = 0.000190.
Element: 2643. Numerical Gradient = 0.000524. BackProp Gradient = 0.000524.
Element: 1048. Numerical Gradient = -0.000631. BackProp Gradient = -0.000631.
Element: 1811. Numerical Gradient = 0.000235. BackProp Gradient = 0.000235.
Element: 1550. Numerical Gradient = -0.000114. BackProp Gradient = -0.000114.
Element: 743. Numerical Gradient = -0.000000. BackProp Gradient = -0.000000.
Element: 1104. Numerical Gradient = 0.000008. BackProp Gradient = 0.000008.
Element: 2779. Numerical Gradient = -0.000068. BackProp Gradient = -0.000068.
Element: 3354. Numerical Gradient = -0.000456. BackProp Gradient = -0.000456.
Element: 1802. Numerical Gradient = 0.000069. BackProp Gradient = 0.000069.


In [9]:
import scipy.optimize
import time
def trainNN(lmbda):
    generatedThetas = initRandomThetas()
    generatedThetas = generatedThetas.reshape(len(generatedThetas),1)
    
    result = scipy.optimize.fmin_cg(costFunction, fprime=backProp, x0=generatedThetas, \
                               args=(Xtrain,yTrain,lmbda),maxiter=50,disp=True,full_output=True)
    return result[0]

start = time.time()
trainThetas = trainNN(0)
end = time.time()
print("Total time is %.2f seconds" %(end - start))

         Current function value: 0.024044
         Iterations: 50
         Function evaluations: 137
         Gradient evaluations: 137
Total time is 34.94 seconds


In [10]:
def predictNN(row, index):
    classes = np.arange(1,11) 
    a3 = aVals[2].T
    a3Row = a3[index]
    rVal = classes[np.argmax(a3Row)]
    return rVal 
    

def propogateForward(row,thetas):
    theta1 = thetas[0:(hiddenSize*(inputSize+1))].reshape(hiddenSize,(inputSize+1))
    theta2 = thetas[(hiddenSize*(inputSize+1)):].reshape(10,(hiddenSize+1))
    
    row = np.insert(row,0,1)
    z2 = theta1.dot(row)
    a2 = expit(z2)
    a2 = np.insert(a2,0,1) #Add the bias unit

    
    
    z3 = theta2.dot(a2)
    a3 = expit(z3)
    
    classes = np.arange(1,11) 
    rVal = classes[np.argmax(a3)]

    return rVal    
    
def computeAccuracy(thetas, X, y):
    m = X.shape[0]
    numCorrect = 0
    #costFunction(thetas, Xtest, yTest, 0)
    for i in range(m):
        if int(propogateForward(X[i],thetas) == int(y[i])):
            numCorrect += 1
    print("Training set accuracy is %.2f" %(100*(numCorrect/m)))
computeAccuracy(trainThetas,Xtest,yTest)

Training set accuracy is 95.07


In [20]:
# redoing nn but using pytorch
import torch
import torch.nn as nn

dataFile = "data/ex4data1.mat"
data = scipy.io.loadmat(dataFile)


# X has 5000 examples and 400 features making it 5000x400
X = data['X']
y = data['y']

X = torch.tensor(X, dtype = torch.float)
y = torch.tensor(y, dtype = torch.float)

In [41]:
class Neural_Network(nn.Module):
    def __init__(self,):
        super(Neural_Network, self).__init__()
        self.inputSize = 400
        self.hiddenSize = 25
        self.outputSize = 10
        
        self.W1 = torch.randn(self.inputSize, self.hiddenSize)
        self.W2 = torch.randn(self.hiddenSize, self.outputSize)
    
    def forward(self, X):
        self.z2 = torch.matmul(X,self.W1)
        self.a2 = expit(self.z2)
        self.z3 = torch.matmul(self.a2, self.W2)
        self.a3 = expit(self.z3)
        
        return self.a3
    
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def backward(self, X, y, out):
        self.out_error = y - out
        self.out_delta = self.out_error * self.sigmoidPrime(out)
        self.z2_error = torch.matmul(self.out_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.out_delta)
    
    def train(self, X, y):
        out = self.forward(X)
        self.backward(X, y, out)
    
    def saveWeights(self, model):
        torch.save(model, "NN")
        
    def predict(self):
        return 1
        
        
        
NN = Neural_Network()
for i in range(400):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
    
NN.saveWeights(NN)
               


#0 Loss: 33.7147102355957
#1 Loss: 38.5
#2 Loss: 38.5
#3 Loss: 38.5
#4 Loss: 38.5
#5 Loss: 38.5
#6 Loss: 38.5
#7 Loss: 38.5
#8 Loss: 38.5
#9 Loss: 38.5
#10 Loss: 38.5
#11 Loss: 38.5
#12 Loss: 38.5
#13 Loss: 38.5
#14 Loss: 38.5
#15 Loss: 38.5
#16 Loss: 38.5
#17 Loss: 38.5
#18 Loss: 38.5
#19 Loss: 38.5
#20 Loss: 38.5
#21 Loss: 38.5
#22 Loss: 38.5
#23 Loss: 38.5
#24 Loss: 38.5
#25 Loss: 38.5
#26 Loss: 38.5
#27 Loss: 38.5
#28 Loss: 38.5
#29 Loss: 38.5
#30 Loss: 38.5
#31 Loss: 38.5
#32 Loss: 38.5
#33 Loss: 38.5
#34 Loss: 38.5
#35 Loss: 38.5
#36 Loss: 38.5
#37 Loss: 38.5
#38 Loss: 38.5
#39 Loss: 38.5
#40 Loss: 38.5
#41 Loss: 38.5
#42 Loss: 38.5
#43 Loss: 38.5
#44 Loss: 38.5
#45 Loss: 38.5
#46 Loss: 38.5
#47 Loss: 38.5
#48 Loss: 38.5
#49 Loss: 38.5
#50 Loss: 38.5
#51 Loss: 38.5
#52 Loss: 38.5
#53 Loss: 38.5
#54 Loss: 38.5
#55 Loss: 38.5
#56 Loss: 38.5
#57 Loss: 38.5
#58 Loss: 38.5
#59 Loss: 38.5
#60 Loss: 38.5
#61 Loss: 38.5
#62 Loss: 38.5
#63 Loss: 38.5
#64 Loss: 38.5
#65 Loss: 38.5
#66 Loss